In [ ]:
# default_exp pod.client
%load_ext autoreload
%autoreload 2

# Pod Client

In [ ]:
# export
from pymemri.data.basic import *
from pymemri.data.schema import *
from pymemri.data.itembase import Edge, ItemBase, Item
from pymemri.data.photo import Photo, NUMPY, BYTES
from pymemri.imports import *
from hashlib import sha256
from pymemri.pod.db import DB
from pymemri.pod.utils import *
from pymemri.plugin.schema import *
from pymemri.test_utils import get_ci_variables

from typing import List, Union
import uuid
import urllib
from datetime import datetime

In [ ]:
# export
DEFAULT_POD_ADDRESS = os.environ.get("POD_ADDRESS") or "http://localhost:3030"
POD_VERSION = "v4"

In [ ]:
# export
class PodClient:
    # Mapping from python type to schema type
    # TODO move to data.schema once schema is refactored
    TYPE_TO_SCHEMA = {
        bool: "Bool",
        str: "Text",
        int: "Integer",
        float: "Real",
        datetime: "DateTime"       
    }

    def __init__(self, url=DEFAULT_POD_ADDRESS, version=POD_VERSION, database_key=None, owner_key=None,
                 auth_json=None, verbose=False, register_base_schema=True):
        self.verbose = verbose
        self.url = url
        self.version = POD_VERSION
        self.test_connection(verbose=self.verbose)

        self.database_key=database_key if database_key is not None else self.generate_random_key()
        self.owner_key=owner_key if owner_key is not None else self.generate_random_key()
        self.base_url = f"{url}/{version}/{self.owner_key}"
        self.auth_json = {"type":"ClientAuth","databaseKey":self.database_key} if auth_json is None \
                          else {**{"type": "PluginAuth"}, **auth_json}

        self.local_db = DB()
        self.registered_classes=dict()
        self.register_base_schemas()
        
    @classmethod
    def from_local_keys(cls, path=DEFAULT_POD_KEY_PATH, **kwargs):
        return cls(database_key=read_pod_key("database_key"), owner_key=read_pod_key("owner_key"), **kwargs)

    @staticmethod
    def generate_random_key():
        return "".join([str(random.randint(0, 9)) for i in range(64)])
    
    def register_base_schemas(self):
        
        assert self.add_to_schema(
            PluginRun, CVUStoredDefinition, Account, Photo
        )

    def test_connection(self, verbose=True):
        try:
            res = requests.get(self.url)
            if verbose: print("Succesfully connected to pod")
            return True
        except requests.exceptions.RequestException as e:
            print("Could no connect to backend")
            return False
        
    def add_to_db(self, node):
            existing = self.local_db.get(node.id)
            if existing is None and node.id is not None:
                self.local_db.add(node)
                
    def reset_local_db(self):
        self.local_db = DB()
        
    def get_create_dict(self, node):
        properties = node.to_json()
        properties = {k:v for k, v in properties.items() if v != []}
        return properties

    def create(self, node):
        create_dict = self.get_create_dict(node)
        try:
            body = {"auth": self.auth_json, "payload": create_dict}

            result = requests.post(f"{self.base_url}/create_item", json=body)
            if result.status_code != 200:
                print(result, result.content)
                return False
            else:
                id = result.json()
                node.id = id
                self.add_to_db(node)
                return True
        except requests.exceptions.RequestException as e:
            print(e)
            return False
        
    def create_photo(self, photo):
        # create the file
        file_succes = self.create_photo_file(photo)
        if file_succes == False:
            raise ValueError("Could not create file")
        # create the photo
        return self.bulk_action(create_items=[photo], create_edges=photo.get_edges("file"))

    
    def _property_dicts_from_instance(self, node):
        create_items = []
        attributes = node.to_json()
        for k, v in attributes.items():
            if type(v) not in self.TYPE_TO_SCHEMA:
                raise ValueError(f"Could not add property {k} with type {type(v)}")
            value_type = self.TYPE_TO_SCHEMA[type(v)]

            create_items.append({
                "type": "ItemPropertySchema", "itemType": attributes["type"],
                "propertyName": k, "valueType": value_type
            })
        return create_items

    
    def _property_dicts_from_type(self, item):
        create_items = []
        for property, p_type in item.get_property_types().items():
            p_type = self.TYPE_TO_SCHEMA[p_type]
            create_items.append({
                "type": "ItemPropertySchema", "itemType": item.__name__,
                "propertyName": property, "valueType": p_type
            })  
        return create_items


    def add_to_schema(self, *items: List[Union[object, type]]):
        create_items = []

        for item in items:
            if isinstance(item, type):
                property_dicts = self._property_dicts_from_type(item)
            else:
                property_dicts = self._property_dicts_from_instance(item)
                item = type(item)
            create_items.extend(property_dicts)
            self.registered_classes[item.__name__] = item

        body = {
            "auth": self.auth_json, "payload": {"createItems": create_items}
        }

        try:
            result = requests.post(f"{self.base_url}/bulk", json=body)
            if result.status_code != 200:
                print(result, result.content)
                return False
        except requests.exceptions.RequestException as e:
            print(e)
            return False
        return True

    def create_photo_file(self, photo):
        file = photo.file[0]
        self.create(file)
        return self._upload_image(photo.data)

    def _upload_image(self, img):
        if isinstance(img, np.ndarray):
            return self.upload_file(img.tobytes())
        elif isinstance(img, bytes):
            return self.upload_file(img)
        else:
            raise ValueError(f"Unknown image data type {type(img)}")

    def upload_file(self, file):
        # TODO: currently this only works for numpy images
        if self.auth_json.get("type") == "PluginAuth":
            # alternative file upload for plugins, with different authentication
            return self.upload_file_b(file)
        else:
            try:
                sha = sha256(file).hexdigest()
                result = requests.post(f"{self.base_url}/upload_file/{self.database_key}/{sha}", data=file)
                if result.status_code in [200, 409]: # 409 = CONFLICT, file already exists
                    return True
                else:
                    print(result, result.content)
                    return False
            except requests.exceptions.RequestException as e:
                print(e)
                return False
            
    def upload_file_b(self, file):
        try:
            sha = sha256(file).hexdigest()
            auth = urllib.parse.quote(json.dumps(self.auth_json))
            result = requests.post(f"{self.base_url}/upload_file_b/{auth}/{sha}", data=file)
            if result.status_code in [200, 409]: # 409 = CONFLICT, file already exists
                return True
            else:
                print(result, result.content)
                return False
        except requests.exceptions.RequestException as e:
            print(e)
            return False        

    def get_file(self, sha):
        # TODO: currently this only works for numpy images
        try:
            body= {"auth": self.auth_json,
                   "payload": {"sha256": sha}}
            result = requests.post(f"{self.base_url}/get_file", json=body)
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                return result.content
        except requests.exceptions.RequestException as e:
            print(e)
            return None

    def get_photo(self, id, size=640):
        photo = self.get(id)
        self._load_photo_data(photo, size=size)
        return photo

    def _load_photo_data(self, photo, size=None):
        if len(photo.file) > 0 and photo.data is None:
            file = self.get_file(photo.file[0].sha256)
            if file is None:
                print(f"Could not load data of {photo} attached file item does not have data in pod")
                return
            if photo.encoding == NUMPY:
                data = np.frombuffer(file, dtype=np.uint8)
                c = photo.channels
                shape = (photo.height,photo.width, c) if c is not None and c > 1 else (photo.height, photo.width)
                data = data.reshape(shape)
                if size is not None: data = resize(data, size)
                photo.data = data
                return
            elif photo.encoding == BYTES:
                photo.data = file
                return
            else:
                raise ValueError("Unsupported encoding")
        print(f"could not load data of {photo}, no file attached")

    def create_if_external_id_not_exists(self, node):
        if not self.external_id_exists(node):
            self.create(node)

    def external_id_exists(self, node):
        if node.externalId is None: return False
        existing = self.search({"externalId": node.externalId})
        return len(existing) > 0

    def create_edges(self, edges):
        """Create edges between nodes, edges should be of format:
           [{"_type": "friend", "_source": 1, "_target": 2}]"""
        return self.bulk_action(create_edges=edges)

#         create_edges = []
#         for e in edges:
#             src, target = e.source.id, e.target.id

#             if src is None or target is None:
#                 print(f"Could not create edge {e} missing source or target id")
#                 return False
#             data = {"_source": src, "_target": target, "_name": e._type}
#             if e.label is not None: data[LABEL] = e.label
#             if e.sequence is not None: data[SEQUENCE] = e.sequence

#             if e.reverse:
#                 data2 = copy(data)
#                 data2["_source"] = target
#                 data2["_target"] = src
#                 data2["_name"] = "~" + data2["_name"]
#                 create_edges.append(data2)

#             create_edges.append(data)

#         return self.bulk_action(create_items=[], update_items=[],create_edges=create_edges)

    def delete_items(self, items):
        return self.bulk_action(delete_items=items) 

    def delete_all(self):
        items = self.get_all_items()
        self.delete_items(items)
        
    @staticmethod
    def gather_batch(items, start_idx, start_size=0, max_size=5000000):
        idx = start_idx
        total_size = start_size
        batch_items = []
        for i, x in enumerate(items):
            if i < idx:
                continue
            elif len(str(x)) > max_size:
                idx = i + 1
                print("Could not add item: Item exceeds max item size")
            elif total_size + len(str(x)) < max_size:
                batch_items.append(x)
                total_size += len(str(x))
                idx = i + 1
            else:
                break
        return batch_items, idx, total_size
    
    def bulk_action(self, create_items=None, update_items=None, create_edges=None, delete_items=None):
        # we need to set the id to not lose the reference
        if create_items is not None:
            for c in create_items:
                if c.id is None: c.id = uuid.uuid4().hex
        create_items = [self.get_create_dict(i) for i in create_items] if create_items is not None else []
        update_items = [self.get_update_dict(i) for i in update_items] if update_items is not None else []
        create_edges = [self.get_create_edge_dict(i) for i in create_edges] if create_edges is not None else []
        # Note: skip delete_items without id, as items that are not in pod cannot be deleted
        delete_items = [item.id for item in delete_items if item.id is not None] if delete_items is not None else []
        
        n_total = len(create_items + update_items + create_edges + delete_items)        
        n=0
        
        i_ci, i_ui, i_ce, i_di = 0,0,0,0
        while not( i_ci == len(create_items) and i_ui == len(update_items) and i_ce == len(create_edges) and i_di == len(delete_items)):
            batch_size=0
            create_items_batch, i_ci, batch_size = self.gather_batch(create_items, i_ci, start_size=batch_size)
            update_items_batch, i_ui, batch_size = self.gather_batch(update_items, i_ui, start_size=batch_size)
            delete_items_batch, i_di, batch_size = self.gather_batch(delete_items, i_di, start_size=batch_size)
            if i_ci == len(create_items): 
                create_edges_batch, i_ce, batch_size = self.gather_batch(create_edges, i_ce, start_size=batch_size)
            else:
                create_edges_batch = []
            n_batch = len(create_items_batch+update_items_batch+create_edges_batch+delete_items_batch)
            n+=n_batch
            print(f"BULK: Writing {n}/{n_total} items/edges")
            succes = self._bulk_action(create_items_batch, update_items_batch, create_edges_batch, delete_items_batch)
            if not succes:
                print("could not complete bulk aciton, aborting")
                return False
        print(f"Completed Bulk action, written {n} items/edges")
        return True

    def _bulk_action(self, create_items=None, update_items=None, create_edges=None, delete_items=None):
        json_data = {
            "auth": self.auth_json,
            "payload": {
                "createItems": create_items, "updateItems": update_items,
                "createEdges": create_edges, "deleteItems": delete_items
            }
        }

        try:
            result = requests.post(f"{self.base_url}/bulk",
                                   json=json_data)
            if result.status_code != 200:
                if "UNIQUE constraint failed" in str(result.content):
                    print(result.status_code, result.content, "Failed bulk update")
                else:
                    print(result, result.content)
                return False
            else:
                return True
        except requests.exceptions.RequestException as e:
            print(e)
            return False

    def get_create_edge_dict(self, edge):
        return {"_source": edge.source.id, "_target": edge.target.id, "_name": edge._type}
        
    def create_edge(self, edge):
        payload = self.get_create_edge_dict(edge)
        body = {"auth": self.auth_json,
                "payload": payload}

        try:
            result = requests.post(f"{self.base_url}/create_edge", json=body)
            if result.status_code != 200:
                print(result, result.content)
                return False
            else:
                return True
        except requests.exceptions.RequestException as e:
            print(e)
            return False

        return self.create_edges([edge])

    def get(self, id, expanded=True):
        if not expanded:
            res = self._get_item_with_properties(id)
        else:
            res = self._get_item_expanded(id)
        if res is None:
            raise ValueError(f"Item with id {id} does not exist")

        elif res.deleted == True:
            print(f"Item with id {id} does not exist anymore")
            return None
        else:
            return res

    def get_all_items(self):
        raise NotImplementedError()
        try:
            body = {  "databaseKey": self.database_key, "payload":None}
            result = requests.post(f"{self.base_url}/get_all_items", json=body)
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                res =  [self.item_from_json(x) for x in json]
                return self.filter_deleted(res)

        except requests.exceptions.RequestException as e:
            print(e)
            return None

    def filter_deleted(self, items):
        return [i for i in items if not i.deleted == True]

    def _get_item_expanded(self, id):
        item = self.get(id, expanded=False)
        edges = self.get_edges(id)
        for e in edges:
            item.add_edge(e["name"], e["item"])
        return item

    def get_edges(self, id):
        body = {"payload": {"item": str(id),
                            "direction": "Outgoing",
                            "expandItems": True},
                "auth": self.auth_json}

        try:
            result = requests.post(f"{self.base_url}/get_edges", json=body)
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                for d in json:
                    d["item"] = self.item_from_json(d["item"])
                return json
        except requests.exceptions.RequestException as e:
            print(e)
            return None

    def _get_item_with_properties(self, id):
        try:
            body = {"auth": self.auth_json,
                    "payload": str(id)}
            result = requests.post(f"{self.base_url}/get_item", json=body)
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                if json == []:
                    return None
                else:
                    res =  self.item_from_json(json[0])
                    return res
        except requests.exceptions.RequestException as e:
            print(e)
            return None
            
    def get_update_dict(self, node):
        properties = node.to_json(dates=False)
        properties.pop("type", None)
        properties.pop("deleted", None)
        return properties
    
    def update_item(self, node):
        data = self.get_update_dict(node)
        body = {"payload": data,
                "auth": self.auth_json}
        try:
            result = requests.post(f"{self.base_url}/update_item",
                                  json=body)
            if result.status_code != 200:
                print(result, result.content)
        except requests.exceptions.RequestException as e:
            print(e)
            
    def exists(self, id):
        try:
            body = {"auth": self.auth_json,
                    "payload": str(id)}
            result = requests.post(f"{self.base_url}/get_item", json=body)
            if result.status_code != 200:
                print(result, result.content)
                return False
            else:
                json = result.json()
                if isinstance(json, list) and len(json) > 0:
                    return True

        except requests.exceptions.RequestException as e:
            print(e)
            return None

    def search_paginate(self, fields_data, limit=50):
        """
        A generator that performs search with pagination.
        
        Returns at least `limit` items, but can return more because
        of pod implementation details.
        """
        if (
            "_limit" in fields_data
            or "dateServerModified" in fields_data
            or "dateServerModified>=" in fields_data
            or "dateServerModified<" in fields_data
        ):
            raise ValueError("Cannot paginate query")
        if "_sortOrder" in fields_data:
            raise NotImplementedError("Only 'Asc' order is supported.")

        response = self.search({**fields_data, "_limit": limit})
        next_dsm = int(response[-1].dateServerModified.timestamp() * 1000) + 1
        yield response

        while True:
            response = client.search(
                {**fields_data, "_limit": limit, "dateServerModified>=": next_dsm}
            )
            if not len(response):
                break
            next_dsm = int(response[-1].dateServerModified.timestamp() * 1000) + 1
            yield response

    def search(self, fields_data, include_edges: bool = True):
        extra_fields = {'[[edges]]': {}} if include_edges else {}
        body = {"payload": {**fields_data, **extra_fields},
                "auth": self.auth_json}
        try:
            result = requests.post(f"{self.base_url}/search", json=body)
            json =  result.json()

            res = [self._item_from_search(item) for item in json]
            return self.filter_deleted(res)
        except requests.exceptions.RequestException as e:
            return None


    def _item_from_search(self, item_json: dict):
        # search returns different fields w.r.t. edges compared to `get` api,
        # different method to keep `self.get` clean.
        item = self.item_from_json(item_json)

        for edge_json in item_json.get("[[edges]]", []):
            edge_name = edge_json["_edge"]
            try:
                edge_item = self.item_from_json(edge_json["_item"])
                item.add_edge(edge_name, edge_item)
            except Exception as e:
                continue
        return item

    def search_last_added(self, type=None, with_prop=None, with_val=None):
        query = {"_limit": 1, "_sortOrder": "Desc"}
        if type is not None:
            query["type"] = type
        if with_prop is not None:
            query[f"{with_prop}=="] = with_val

        return self.search(query)[0]

    def item_from_json(self, json):
        plugin_class = json.get("pluginClass", None)
        plugin_package = json.get("pluginPackage", None)

        constructor = get_constructor(json["type"], plugin_class, plugin_package=plugin_package,
                                      extra=self.registered_classes)
        new_item = constructor.from_json(json)
        existing = self.local_db.get(new_item.id)
        # TODO: cleanup
        if existing is not None:
            if not existing.is_expanded() and new_item.is_expanded():
                for edge_name in new_item.get_all_edge_names():
                    edges = new_item.get_edges(edge_name)
                    for e in edges:
                        e.source = existing
                    existing.__setattr__(edge_name, edges)

            for prop_name in new_item.get_property_names():
                existing.__setattr__(prop_name, new_item.__getattribute__(prop_name))
            return existing
        else:
            return new_item

    def get_properties(self, expanded):
        properties = copy(expanded)
        if ALL_EDGES in properties: del properties[ALL_EDGES]
        return properties
    
    def send_email(self, to, subject="", body=""):
        body = {"payload": {"to": to,
                            "subject": subject,
                            "body": body
                           },
                "auth": self.auth_json}
        try:
            result = requests.post(f"{self.base_url}/send_email", json=body)
            if result.status_code == 200:
                print(f"succesfully sent email to {to}")
            else:
                print(f"could not send email {result.content}")
        except requests.exceptions.RequestException as e:
            print(e)
            return None

Pymemri communicates with the pod via the `PodClient`. The PodClient requires you to provide a [database key](https://gitlab.memri.io/memri/pod/-/blob/dev/docs/HTTP_API.md#user-content-api-authentication-credentials) and an [owner key](https://gitlab.memri.io/memri/pod/-/blob/dev/docs/HTTP_API.md#user-content-api-authentication-credentials). During development, you don't have to worry about these keys, you can just omit the keys when initializing the `PodClient`, which creates a new user by defining random keys. *Note that this will create a new database for your every time you create a PodClient, if you want to access the same database with multiple PodClients, you have to set the  same keys* When you are using the app, setting the keys in the pod, and passing them when calling an integrator is handled for you by the app itself.

In [ ]:
client = PodClient()
success = client.test_connection()
assert success

Succesfully connected to pod


## Creating Items and Edges

Now that we have access to the pod, we can create items here and upload them to the pod. All items are defined in the schema of the pod. To create an item in the pod, you have to add the schema first. Schemas can be added as follows

In [ ]:
from pymemri.data.schema import EmailMessage, Address, PhoneNumber

succes = client.add_to_schema(EmailMessage, Address, PhoneNumber)
assert succes

We can now create our item. As a side-effect, our item will be assigned an id.

In [ ]:
email_item = EmailMessage.from_data(content="example content field")
client.create(email_item)

True

In [ ]:
email_item.id

'e1c9f75fd6e46f4bb43cb6c6d00f4fe9'

We can easily define our own types, and use them in the pod.

In [ ]:
# export
class Dog(Item):
    properties = Item.properties + ["name", "age", "bites", "weight"]
    edges = Item.edges 
    def __init__(self, name: str=None, age: int=None, bites: bool=False, weight: float=None, **kwargs):
        super().__init__(**kwargs)
        self.name = name
        self.age = age
        self.bites = bites
        self.weight = weight

In [ ]:
dog = Dog("", 0, True, 0.)
client.add_to_schema(dog)
dog2 = Dog(name="bob", age=3, weight=33.2)
client.create(dog2);

In [ ]:
client.reset_local_db()
dog_from_db = client.get(dog2.id)

In [ ]:
assert dog_from_db.name == "bob"
assert dog_from_db.age == 3
assert dog_from_db.weight == 33.2

We can connect items using edges. Let's create another item, a person, and connect the email and the person.

In [ ]:
person_item = Person.from_data(firstName="Alice", lastName="X")
succes = client.add_to_schema(person_item)
assert succes

In [ ]:
person_item = Person.from_data(firstName="Alice", lastName="X")
item_succes = client.create(person_item)
edge = Edge(email_item, person_item, "sender")
edge_succes = client.create_edge(edge)
assert item_succes
assert edge_succes

In [ ]:
client.get_edges(email_item.id)

[{'item': Person (#988e4015b1add71cf04ed7ad8d849632), 'name': 'sender'}]

If we use the normal `client.get` (without `expanded=False`), we also get items directly connected to the Item.

In [ ]:
email_from_db = client.get(email_item.id) 

In [ ]:
assert isinstance(email_from_db.sender[0], Person)

# Fetching and updating Items

## Normal Items

We can use the client to fetch data from the database. This is in particular useful for indexers, which often use data in the database as input for their models. The simplest form  of querying the database is by querying items in the pod by their id (unique identifier).

In [ ]:
person_item = Person.from_data(firstName="Alice")
assert client.create(person_item)

In [ ]:
person_from_db = client.get(person_item.id, expanded=False)
assert person_from_db is not None
assert person_from_db == person_item
assert person_from_db.id is not None

Appart from creating, we might want to update existing items:

In [ ]:
person_item.lastName = "Awesome"
client.update_item(person_item)
person_from_db = client.get(person_item.id, expanded=False)
assert person_from_db.lastName == "Awesome"

When we don't know the ids of the items we want to fetch, we can also search by property. We can use this for instance when we want to query all items from a particular type to perform some indexing on. We can get all `Person` Items from the db by:

## Search

In [ ]:
person_item2 = Person.from_data(firstName="Bob")
person_account = Account(service="testService")
client.create(person_item2)
client.create(person_account)

person_item2.add_edge("account", person_account)
client.create_edges(person_item2.get_edges("account"))

BULK: Writing 1/1 items/edges
Completed Bulk action, written 1 items/edges


True

In [ ]:
# search without edges

# To test search with and without include_edges, make a fresh client (no local_db)
new_client = PodClient(owner_key=client.owner_key, database_key=client.database_key)

all_people = new_client.search({"type": "Person"}, include_edges=False)
assert all([isinstance(p, Person) for p in all_people]) and len(all_people) > 0
assert all([len(p.account)==0 for p in all_people])

In [ ]:
# Search with edges

all_people = new_client.search({"type": "Person"}, include_edges=True)
assert all([isinstance(p, Person) for p in all_people]) and len(all_people) > 0
assert any([len(p.account) for p in all_people])

In [ ]:
# Search with pagination

for i in range(10):
    client.bulk_action(create_items=[
        Account(identifier=f"{j + i*10}", service="paginate_test") for j in range(10)
    ])

accounts = client.search({"type": "Account", "service": "paginate_test"})

generator = client.search_paginate({"type": "Account", "service": "paginate_test"})
accounts_paginated = []
for page in generator:
    accounts_paginated.extend(page)
    
assert len(accounts_paginated) == 100
assert [a.id for a in accounts] == [a.id for a in accounts_paginated]

## Search last added items

In [ ]:
person_item2 = Person.from_data(firstName="Last Person")
client.create(person_item2);

In [ ]:
assert client.search_last_added(type="Person").firstName == "Last Person"

In the near future, Pod will support searching by user defined properties as well. This will allow for the following. **warning, this is currently not supported**

```client.search_last_added(type="Person", with_prop="ImportedBy", with_val="EmailImporter")```

## Uploading & downloading files

### File API

To work with files, the `PodClient` has a file api. The file api works by posting a blob to the `upload_file` endpoint, and creating an Item with a property with the same sha256 as the sha used in the endpoint.

In [ ]:
from pymemri.data.photo import *

In [ ]:
x = np.random.randint(0, 255+1, size=(640, 640), dtype=np.uint8)
photo = Photo.from_np(x)
file = photo.file[0]
succes = client.create(file)
succes2 = client._upload_image(x)
assert succes
assert succes2

In [ ]:
data = client.get_file(file.sha256)
arr = np.frombuffer(data, dtype=np.uint8)
assert (arr.reshape(640,640) == x).all()

### Photo API

For photos we do this automatically using `PodClient.create` on a Photo and `PodClient.get_photo`:

In [ ]:
x = np.random.randint(0, 255+1, size=(640, 640), dtype=np.uint8)
photo = Photo.from_np(x)

In [ ]:
succes = client.add_to_schema(Photo.from_np(x))

In [ ]:
client.create_photo(photo)

BULK: Writing 2/2 items/edges
Completed Bulk action, written 2 items/edges


True

In [ ]:
res = client.get_photo(photo.id, size=640)

In [ ]:
assert (res.data == x).all()

Some photos come as bytes, we can use `Iphoto.from_bytes` to initialize those

In [ ]:
_bytes = b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xe1\x00\x00\x00\xe1\x08\x03\x00\x00\x00\tm"H\x00\x00\x003PLTE\x04\x02\x04\x00\x00\x00\xa0\xa0\xa0\xa3\xa3\xa3\xaa\xaa\xaa\xb4\xb4\xb4\xbd\xbe\xbd\xbb\xbc\xbb\xde\xde\xde\x9b\x9a\x9b\xfe\xfe\xfe\xf2\xf3\xf2\xe5\xe6\xe5\xd8\xd9\xd8\xd1\xd1\xd1\xc9\xca\xc9\xae\xae\xae\x80k\x98\xfc\x00\x00\x01TIDATx\x9c\xed\xdd;r\xc2P\x00\x04A!\x90\x84\xfc\x01\xee\x7fZ\x138\xb1\x13S\xceF\xaf\xfb\x06\x93o\xd5No\xef\x1f\x9f\xb7\xfb}]\xd7my\xba|;\xff4\xff\xdf\xf9O\x97W<\x96W\xac\xbfm\xd7i9\x1d\xdb\xfe,\x9c\x8e\xec4+\xac{\x16^\x14\xb6)\xecS\xd8\xa7\xb0Oa\x9f\xc2\xbe!\n\xcf\n\xdb\x14\xf6)\xecS\xd8\xa7\xb0Oa\x9f\xc2>\x85}\n\xfb\x14\xf6)\xecS\xd8\xa7\xb0Oa\x9f\xc2>\x85}C\x14\xce\n\xdb\x14\xf6)\xecS\xd8\xa7\xb0Oa\x9f\xc2>\x85}\n\xfb\x14\xf6)\xecS\xd8\xa7\xb0Oa\x9f\xc2>\x85}\n\xfb\x14\xf6)\xecS\xd87\xc4bHa\x9c\xc2>\x85}\n\xfb\x14\xf6)\xecS\xd8\xa7\xb0Oa\x9f\xc2>\x85}\n\xfb\x14\xf6)\xecS\xd8\xa7\xb0Oa\x9f\xc2>\x85}\n\xfb\x86xaQ\x18\xa7\xb0Oa\x9f\xc2>\x85}\n\xfb\x14\xf6)\xecS\xd8\xa7\xb0Oa\x9f\xc2>\x85}\n\xfb\x14\xf6)\xecS\xd87D\xe1\xe3\xf0\x85\x8b\xc26\x85}\n\xfb\x14\xf6)\xecS\xd8\xa7\xb0Oa\x9f\xc2>\x85}\n\xfb\x14\xf6)\xecS\xd8\xa7\xb0Oa\x9f\xc2>\x85}C\x14\xae\n\xdb\x14\xf6)\xecS\xd8\xa7\xb0Oa\x9f\xc2>\x85}C\x14n\xa7c\xdb\xa7\xeb>\x1f\xd9~\xfb\x02\xee\x7f\r\xe5\xe1h\x04"\x00\x00\x00\x00IEND\xaeB`\x82'

In [ ]:
photo = Photo.from_bytes(_bytes)

In [ ]:
assert client.create_photo(photo)

BULK: Writing 2/2 items/edges
Completed Bulk action, written 2 items/edges


We create a new client to prevent caching

In [ ]:
new_client = PodClient(database_key=client.database_key, owner_key=client.owner_key)

In [ ]:
res = new_client.get_photo(photo.id, size=225)

In [ ]:
res.data == photo.data

True

In [ ]:
res.file

[File (#f97f96d594a36ba33bbece3dba8a62ac)]

## Bulk API

Currently, an api call takes a lot of time (~0.1 second). If you want to write many data items to the pod, you can use the bulk api for efficiency reasons. Currently, only creating `Items` and `Edges` is supported, support for updating and deletion will follow.

In [ ]:
# Test bulk create items and edges

dogs = [Dog(name=f"dog number {i} " +"a" * 10000) for i in range(500)]
person = Person(firstName="Alice")
edge1 = Edge(dogs[0], person, "label")
edge2 = Edge(dogs[1], person, "label")

client.bulk_action(create_items=dogs + [person], create_edges=[edge1,edge2])

dogs_with_edge = [item for item in client.search({"type": "Dog"}) if item.name.startswith("dog number 0")
                  or item.name.startswith("dog number 1 ")]

assert len(dogs_with_edge) == 2
for d in dogs_with_edge:
    assert len(d.label) > 0

BULK: Writing 495/503 items/edges
BULK: Writing 503/503 items/edges
Completed Bulk action, written 503 items/edges


In [ ]:
 # test bulk delete and update

# Change person name, delete first dog :(
person.firstName = "Bob"
to_delete = [dogs[0]]
to_update = [person]

client.bulk_action(delete_items=to_delete, update_items=to_update)
dogs_with_edge = [
    item for item in client.search({"type": "Dog"}) if item.name.startswith("dog number 0") or item.name.startswith("dog number 1 ")
]

assert len(dogs_with_edge) == 1

dog = dogs_with_edge[0]
assert dog.label[0].firstName == "Bob"

BULK: Writing 2/2 items/edges
Completed Bulk action, written 2 items/edges


# Sending emails -

In [ ]:
if False:
    to = "myemail@gmail.com"
    client.send_email(to=to, subject="test", body="test2")

# Check if an item exists -

In [ ]:
# hide
# person_item = Person.from_data(firstName="Eve",   externalId="gmail_1")
# person_item2 = Person.from_data(firstName="Eve2", externalId="gmail_1")

# client.create_if_external_id_not_exists(person_item)
# client.create_if_external_id_not_exists(person_item2)

# existing = client.search({"externalId": "gmail_1"})
# assert len(existing) == 1
# client.delete_all()

# Resetting the db -

In [ ]:
# client.delete_all()

# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted Untitled.ipynb.
Converted basic.ipynb.
Converted cvu.utils.ipynb.
Converted data.photo.ipynb.
Converted index.ipynb.
Converted itembase.ipynb.
Converted plugin.authenticators.credentials.ipynb.
Converted plugin.authenticators.oauth.ipynb.
Converted plugin.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.states.ipynb.
Converted plugins.authenticators.password.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
Converted test_schema.ipynb.
Converted test_utils.ipynb.
